In [1]:
print("hello")

hello


In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install re

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
import re
from datetime import datetime, timedelta

In [6]:
keywords_df = pd.read_csv('C:/Users/GurnoorSingh/OneDrive - ProcDNA Analytics Pvt. Ltd/Internal/Auxo/Keywords_df.csv')
posts_df= pd.read_csv('C:/Users/GurnoorSingh/OneDrive - ProcDNA Analytics Pvt. Ltd/Internal/Auxo/Posts_df.csv', encoding='ISO-8859-1')

In [7]:
posts_df.columns = posts_df.columns.str.strip()
keywords_df.columns = keywords_df.columns.str.strip()


In [8]:
# Convert keywords and responses into a dictionary
keywords_to_responses = dict(zip(keywords_df['Keyword'], keywords_df['Response']))

In [9]:
# Convert keywords and priorities into a dictionary
keywords_to_priority = dict(zip(keywords_df['Keyword'], keywords_df['Keyword_Priority']))

In [10]:
# Define your medical keywords
medical_keywords = ['lymphoma', 'cancer', 'oncology'] 

In [11]:
# Function to filter posts with medical keywords
def filter_medical_keywords(post_text):
    post_text = str(post_text).lower()
    return any(med_keyword in post_text for med_keyword in medical_keywords)

In [12]:
# Filter posts based on medical keywords
filtered_posts_df = posts_df[posts_df['Post_Text'].apply(filter_medical_keywords)]

In [13]:
# Define priority ranges
priority_days = {
    1: 7,
    2: 14,
    3: 30,
    4: 60,
    5: 90,
    6: 180,
    7: 365,
    8: 730,
    9: 1095,
    10: float('inf')  # Priority 10 for posts older than 3 years
}

In [14]:
datetime.now()

datetime.datetime(2024, 9, 27, 11, 53, 9, 921009)

In [15]:
# Function to calculate date priority
def calculate_date_priority(post_date_str, reference_date=None):
    if reference_date is None:
        reference_date = datetime.now()
    
    try:
        post_date = datetime.strptime(post_date_str, '%m-%d-%Y')
        days_diff = (reference_date - post_date).days
        
        for priority, days in sorted(priority_days.items()):
            if days_diff <= days:
                return priority
    except ValueError:
        # Handle date parsing errors
        return 10  # Default priority if date is not valid

    return 10  # Default priority for posts older than the maximum specified

In [16]:
# Function to extract a snippet of the context around the keyword
def extract_keyword_context(post_text, keyword, word_limit=15):
    sentences = re.split(r'(?<=[.!?]) +', post_text)
    keyword_sentences = [sentence for sentence in sentences if keyword.lower() in sentence.lower()]
    
    hashtags = re.findall(r'#\w+', post_text)
    top_hashtags = hashtags[:2]
    
    snippets = []
    for sentence in keyword_sentences:
        words = sentence.split()
        keyword_index = next((i for i, word in enumerate(words) if keyword.lower() in word.lower()), None)
        if keyword_index is not None:
            start = max(keyword_index - (word_limit // 2), 0)
            end = min(keyword_index + (word_limit // 2) + 1, len(words))
            snippet = ' '.join(words[start:end])
            if len(words) > word_limit:
                snippet += "..."
            snippets.append(snippet)
    
    context = ' '.join(snippets)
    hashtags_str = ", ".join(top_hashtags)
    
    return context, hashtags_str

In [17]:
# Function to check for keywords in a post and return the corresponding response
def check_post_for_keywords(row):
    post_text = str(row.get('Post_Text', ''))
    source = row.get('Source', '')
    hcp_name = row.get('HCP_Name', '')
    post_url = row.get('Post_URL', '')
    post_date = row.get('Post_Date', '')

    # Ensure post_text is a string
    if pd.isna(post_text):
        post_text = ''
    else:
        post_text = str(post_text)
    
    for keyword, response in keywords_to_responses.items():
        if keyword.lower() in post_text.lower():
            context, hashtags_str = extract_keyword_context(post_text, keyword)
            keyword_priority = keywords_to_priority.get(keyword, 10)  # Default priority is 10 if not found
            date_priority = calculate_date_priority(post_date)
            
            # Average the keyword priority and date priority
            final_priority = (keyword_priority + date_priority) / 2 #Final priority has been kept as average for now.
            
            if hashtags_str:
                full_response = f'{hcp_name} has posted on {source} about "{context}" and "{hashtags_str}". [Click here]({post_url})'
            else:
                full_response = f'{hcp_name} has posted on {source} about "{context}". [Click here]({post_url})'
            return full_response, keyword, keyword_priority, date_priority, final_priority, post_date  # Include post_date here
    
    return None, None, None, None, None, post_date  # Ensure post_date is returned even if no keyword is found

In [18]:
# Apply the keyword detection and priority function to each filtered post
try:
    filtered_posts_df[['Keyword_Response', 'Detected_Keyword', 'Keyword_Priority', 'Date_Priority', 'Final_Priority', 'Post_Date']] = filtered_posts_df.apply(
        lambda row: pd.Series(check_post_for_keywords(row)), axis=1
    )
except KeyError as e:
    print(f"Error applying function: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

C:\Users\GurnoorSingh\AppData\Local\Temp\ipykernel_132384\1423970787.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_posts_df[['Keyword_Response', 'Detected_Keyword', 'Keyword_Priority', 'Date_Priority', 'Final_Priority', 'Post_Date']] = filtered_posts_df.apply(
C:\Users\GurnoorSingh\AppData\Local\Temp\ipykernel_132384\1423970787.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_posts_df[['Keyword_Response', 'Detected_Keyword', 'Keyword_Priority', 'Date_Priority', 'Final_Priority', '

In [19]:
# Further filter out posts without detected keywords
final_filtered_posts_df = filtered_posts_df.dropna(subset=['Detected_Keyword'])

In [21]:
# Display the results
try:
    from IPython.display import display
    display(final_filtered_posts_df[['NPI_ID', 'HCP_Name', 'Source', 'Post_Date', 'Keyword_Response', 'Final_Priority']])
except ImportError:
    # If IPython is not available, display the DataFrame in a terminal
    print(final_filtered_posts_df[['NPI_ID', 'HCP_Name', 'Source', 'Post_Date', 'Keyword_Response', 'Final_Priority']])

,NPI_ID,HCP_Name,Source,Post_Date,Keyword_Response,Final_Priority
0,1265468110,Andrew M Evens,LinkedIn,05-08-2024,"Andrew M Evens has posted on LinkedIn about ""I...",4.0
1,1578618161,Naveen Pemmaraju,Twitter,05-16-2024,"Naveen Pemmaraju has posted on Twitter about ""...",3.5
2,1992892079,John Paul Leonard,Twitter,05-17-2024,John Paul Leonard has posted on Twitter about ...,5.0
5,1669667028,Binay Kumar Shah,LinkedIn,05-13-2024,Binay Kumar Shah has posted on LinkedIn about ...,4.0
8,1265468110,Andrew M Evens,Twitter,05-06-2024,"Andrew M Evens has posted on Twitter about ""su...",5.0
11,1265468110,Andrew M Evens,LinkedIn,05-09-2024,"Andrew M Evens has posted on LinkedIn about ""a...",6.5
13,1265468110,Andrew M Evens,LinkedIn,05-03-2024,"Andrew M Evens has posted on LinkedIn about ""W...",6.5
14,1265468110,Andrew M Evens,LinkedIn,05-03-2024,"Andrew M Evens has posted on LinkedIn about ""S...",6.5
15,1265468110,Andrew M Evens,LinkedIn,05-04-2024,"Andrew M Evens has posted on LinkedIn about ""G...",5.0
16,1265468110,Andrew M Evens,LinkedIn,05-05-2024,"Andrew M Evens has posted on LinkedIn about ""a...",5.0


In [22]:
# Sort the final filtered posts based on ascending order of Final_Priority
final_filtered_posts_df = final_filtered_posts_df.sort_values(by='Final_Priority', ascending=True)

# Display the sorted results
try:
    from IPython.display import display
    display(final_filtered_posts_df[['NPI_ID', 'HCP_Name', 'Source', 'Post_Date', 'Keyword_Response', 'Final_Priority']])
except ImportError:
    print(final_filtered_posts_df[['NPI_ID', 'HCP_Name', 'Source', 'Post_Date', 'Keyword_Response', 'Final_Priority']])


,NPI_ID,HCP_Name,Source,Post_Date,Keyword_Response,Final_Priority
1,1578618161,Naveen Pemmaraju,Twitter,05-16-2024,"Naveen Pemmaraju has posted on Twitter about ""...",3.5
0,1265468110,Andrew M Evens,LinkedIn,05-08-2024,"Andrew M Evens has posted on LinkedIn about ""I...",4.0
5,1669667028,Binay Kumar Shah,LinkedIn,05-13-2024,Binay Kumar Shah has posted on LinkedIn about ...,4.0
2,1992892079,John Paul Leonard,Twitter,05-17-2024,John Paul Leonard has posted on Twitter about ...,5.0
8,1265468110,Andrew M Evens,Twitter,05-06-2024,"Andrew M Evens has posted on Twitter about ""su...",5.0
15,1265468110,Andrew M Evens,LinkedIn,05-04-2024,"Andrew M Evens has posted on LinkedIn about ""G...",5.0
16,1265468110,Andrew M Evens,LinkedIn,05-05-2024,"Andrew M Evens has posted on LinkedIn about ""a...",5.0
11,1265468110,Andrew M Evens,LinkedIn,05-09-2024,"Andrew M Evens has posted on LinkedIn about ""a...",6.5
14,1265468110,Andrew M Evens,LinkedIn,05-03-2024,"Andrew M Evens has posted on LinkedIn about ""S...",6.5
13,1265468110,Andrew M Evens,LinkedIn,05-03-2024,"Andrew M Evens has posted on LinkedIn about ""W...",6.5


In [26]:
# Ensure 'NPI_ID' column is string type for consistent filtering
final_filtered_posts_df['NPI_ID'] = final_filtered_posts_df['NPI_ID'].astype(str)

# Function to filter results by a specific NPI_ID
def filter_by_npi(final_filtered_posts_df, npi_id):
    # Ensure the provided NPI_ID is a string
    npi_id = str(npi_id)
    
    # Filter the DataFrame by the provided NPI_ID
    filtered_by_npi_df = final_filtered_posts_df[final_filtered_posts_df['NPI_ID'] == npi_id]
    
    # If no results are found for the NPI_ID
    if filtered_by_npi_df.empty:
        print(f"No results found for NPI_ID: {npi_id}")
    else:
        try:
            from IPython.display import display
            display(filtered_by_npi_df[['NPI_ID', 'HCP_Name', 'Source', 'Post_Date', 'Keyword_Response', 'Final_Priority']])
        except ImportError:
            print(filtered_by_npi_df[['NPI_ID', 'HCP_Name', 'Source', 'Post_Date', 'Keyword_Response', 'Final_Priority']])

# Example: Set the desired NPI_ID (you can replace this with any valid NPI_ID)
npi_id = '1265468110'  # Replace with your target NPI_ID

# Call the function to filter and display results for the given NPI_ID
filter_by_npi(final_filtered_posts_df, npi_id)


,NPI_ID,HCP_Name,Source,Post_Date,Keyword_Response,Final_Priority
0,1265468110,Andrew M Evens,LinkedIn,05-08-2024,"Andrew M Evens has posted on LinkedIn about ""I...",4.0
8,1265468110,Andrew M Evens,Twitter,05-06-2024,"Andrew M Evens has posted on Twitter about ""su...",5.0
15,1265468110,Andrew M Evens,LinkedIn,05-04-2024,"Andrew M Evens has posted on LinkedIn about ""G...",5.0
16,1265468110,Andrew M Evens,LinkedIn,05-05-2024,"Andrew M Evens has posted on LinkedIn about ""a...",5.0
11,1265468110,Andrew M Evens,LinkedIn,05-09-2024,"Andrew M Evens has posted on LinkedIn about ""a...",6.5
14,1265468110,Andrew M Evens,LinkedIn,05-03-2024,"Andrew M Evens has posted on LinkedIn about ""S...",6.5
13,1265468110,Andrew M Evens,LinkedIn,05-03-2024,"Andrew M Evens has posted on LinkedIn about ""W...",6.5


In [18]:
# Save the updated DataFrame to a new CSV file
final_filtered_posts_df.to_csv('C:/Users/GurnoorSingh/OneDrive - ProcDNA Analytics Pvt. Ltd/Internal/Auxo/Outputs.csv')